# Using the Script-Language Container
A [Script-Language Container](https://github.com/exasol/script-languages-release) for the Exasol database consists of a Linux container with a complete Linux distribution and all required libraries, such as a script client. The script client is responsible for the communication with the  database and for executing the script code. It allows to also include user specific libraries which can then be used from within the UDF's. 

Before we start we need to configure the script-languages directory and flavor. See [Configure SLC Flavor](http://localhost:49494/lab/tree/script_languages_container/configure_slc_flavor.ipynb)

## Setup
### Open Secure Configuration Storage

In [44]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

Output()

Box(children=(Box(children=(Label(value='Configuration Store', layout=Layout(border_bottom='solid 1px', border…

### Utility functions

#### Working directory context manager
Let's create a utility context manager function which enters a directory and jumps back to the previous directory on exit.

In [45]:
import os
import contextlib
from pathlib import Path

@contextlib.contextmanager
def working_directory(path: Path):
    """Changes working directory and returns to previous on exit."""
    prev_cwd = Path.cwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(prev_cwd)

#### Show directory content

In [56]:
def show_directory_content(directory: Path):
    for p in directory.iterdir():
        print(p)

### Import some utility functions

In [46]:
from exasol.nb_connector.utils import upward_file_search

# This NB may be running from various locations in the NB hierarchy.
# Need to search for the styles NB from the current directory upwards.

%run {upward_file_search('utils/ui_styles.ipynb')}
%run {upward_file_search('utils/popup_message_ui.ipynb')}

Output()

## Let's use the flavor as is
### Check selected flavor path

In [50]:
from pathlib import Path
slc_source_flavor_store_key = "slc_flavor"
slc_target_dir_store_key = "slc_target_dir"

slc_dir = Path(ai_lab_config.get(slc_target_dir_store_key))
selected_flavor = ai_lab_config.get(slc_source_flavor_store_key)
print(f"Script-languages repository path is '{slc_dir}'")
print(f"Selected flavor is '{selected_flavor}'")
if not (Path(slc_dir) / "flavors" / selected_flavor).is_dir():
    popup_message("Invalid script-languages directory or flavor configuration. Please re-run ")

flavor_path = Path("flavors") / selected_flavor


Script-languages repository path is '/tmp/script_languages_release'
Selected flavor is 'template-Exasol-all-python-3.10'


### Export the flavor
You could run the same on the command line with the `exaslct` tool:
```
cd <slc_dir>
./exaslct export --flavor-path flavors/<flavor_path>
```

In [52]:
from exasol_script_languages_container_tool.lib import api as exaslct_api

export_path = "/tmp/slc"
output_path = "/tmp/output"

with working_directory(slc_dir):
    exaslct_api.export(flavor_path=(str(flavor_path),), export_path=export_path)
print("Ready")

Ready


#### Check the result

In [57]:
show_directory_content(Path(export_path))

/tmp/slc/template-Exasol-all-python-3.10_release.tar.gz
/tmp/slc/template-Exasol-all-python-3.10_release.tar.gz.sha512sum
